## Importações

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob


## Entendendo os dados

### Arquivos de treino
Contem dados de interações dos usuários com as notícias antes de uma data limite T. Incluem informações sobre quais notícias foram acessadas, quando, por quanto tempo, quantos cliques o usuário deu, etc.

In [3]:
treino_parte1 = pd.read_csv('/Users/leticiapires/Desktop/G1Recommender/dados/challenge-webmedia-e-globo-2023/files/treino/treino_parte1.csv')
treino_parte1.head(2)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...","1656684240278, 1656761266729, 1656761528085, 1...","7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13, ...","6049, 210489, 8672, 10000, 30000, 123007, 9965...","25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.33...","1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1...","1656684240278, 1656761266729, 1656761528085, 1..."


O que significa cada coluna?

- `userId`: ID único do usuário.
- `userType`: Se o usuário está logado ou é anônimo.
- `HistorySize`: Quantidade total de notícias lidas pelo usuário.
- `history`: Lista das notícias visitadas pelo usuário (IDs das páginas).
- `TimestampHistory`: Momento em que cada notícia foi lida.
- `timeOnPageHistory`: Tempo (em milissegundos) que o usuário ficou na página.
- `numberOfClicksHistory`: Quantidade de cliques feitos na notícia.
- `scrollPercentageHistory`: Percentual da página visualizada pelo usuário.
- `pageVisitsCountHistory`: Quantidade de vezes que o usuário visitou a mesma notícia.

### Itens

In [4]:
itens = pd.read_csv('/Users/leticiapires/Desktop/G1Recommender/dados/challenge-webmedia-e-globo-2023/itens/itens/itens-parte1.csv')
itens.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


`Itens` contêm informações sobre as notícias que aparecem no histórico dos usuários. 

O que significa cada coluna?

- `Page`: ID da notícia, que aparece na coluna history do conjunto de treino.
- `Url`: Link da matéria no Globoplay.
- `Issued`: Data de publicação da notícia.
- `Modified`: Última data em que a matéria foi modificada.
- `Title`: Título da matéria.
- `Body`: Texto completo da notícia.
- `Caption`: Subtítulo da notícia.


#### Como os dados se relacionam?

Os dois conjuntos de dados estão ligados pelo ID da notícia (Page), que aparece tanto no treino (history) quanto nos itens (Page).


### Validação

In [5]:
validacao = pd.read_csv('/Users/leticiapires/Desktop/G1Recommender/dados/challenge-webmedia-e-globo-2023/validacao.csv')
validacao.head()

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8'\n '01c...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]
2,755062dd39a48809880cf363b04268c3af2c003088cde0...,Logged,['857aa90f-a7ec-410d-ba82-dfa4f85d4e71'],[1660561649242]
3,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,Logged,['b7b90e18-7613-4ca0-a8fc-fd69addfcd85'\n '835...,[1660533830245 1660540831707 1660542659111 166...
4,a120515626fe5d12b22b7d5a7c5008912cc69284aa26cc...,Logged,['9c764c3a-f9f8-4fb2-b2c4-6331eaeb3dd6'\n 'b8e...,[1660548813953 1660572329731 1660594848200]


O conjunto de validação contém um período de tempo posterior ao conjunto de treino. Ou seja, ele contém os usuários que já estão no treino, mas mostra um novo conjunto de interações que ocorreram após T.

Colunas do conjunto de validação:

- `userId`: Identificação do usuário.
- `userType`: Usuário logado ou anônimo.
- `history`: Lista de notícias que o usuário visitou após T.

Ou seja, temos quais notícias o usuário acessou depois de T.

### Analisar coluna de timestamp

Tem uma lista de timestamps, isso representa as datas que um uma pessoa visitou a mesma noticia?

# Juntando tabelas

In [6]:

# Listar e carregar os arquivos de treino
treino_files = glob.glob('/Users/leticiapires/Desktop/G1Recommender/dados/challenge-webmedia-e-globo-2023/files/treino/treino_parte*.csv')
df_treino = pd.concat([pd.read_csv(f) for f in treino_files], ignore_index=True)

In [7]:
df_treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577942 entries, 0 to 577941
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   userId                   577942 non-null  object
 1   userType                 577942 non-null  object
 2   historySize              577942 non-null  int64 
 3   history                  577942 non-null  object
 4   timestampHistory         577942 non-null  object
 5   numberOfClicksHistory    577942 non-null  object
 6   timeOnPageHistory        577942 non-null  object
 7   scrollPercentageHistory  577942 non-null  object
 8   pageVisitsCountHistory   577942 non-null  object
 9   timestampHistory_new     577942 non-null  object
dtypes: int64(1), object(9)
memory usage: 44.1+ MB


In [8]:
df_treino.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,Non-Logged,2,"80aa7bb2-adce-4a55-9711-912c407927a1, d9e5f15d...","1657908085200, 1659634203762","0, 0","71998, 115232","81.58, 73.36","1, 1","1657908085200, 1659634203762"
1,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,Non-Logged,2,"19ba89fc-1e06-4c5d-9c57-4a3088dc0511, e273dba4...","1657111508570, 1657481309920","68, 12","131495, 43733","51.74, 35.49","1, 1","1657111508570, 1657481309920"
2,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,Non-Logged,2,"59a61a8a-cc52-453f-b1cd-2bd019e9d574, a0562805...","1657823890328, 1660141444328","55, 9","159042, 10336","62.19, 48.28","1, 1","1657823890328, 1660141444328"
3,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,Non-Logged,2,"233f8238-2ce0-470f-a9d5-0e0ac530382a, 037155f4...","1656963373076, 1657091888917","0, 0","193579, 20519","31.03, 31.9","1, 1","1656963373076, 1657091888917"
4,97e1439d485b0630e12818d3df84ff67d08475ef6ebeb0...,Logged,2,"385044ad-3876-4188-83fa-f560435c1a9c, 2f754502...","1657618607633, 1659536839832","57, 38","220000, 130000","52.65, 53.37","1, 1","1657618607633, 1659536839832"


In [9]:
# import ast

# def force_list_conversion(value):
#     """Garante que os valores são listas antes da explosão."""
#     if isinstance(value, str):
#         try:
#             return ast.literal_eval(value)  # Converte a string para lista real
#         except:
#             return [value]  # Se der erro, transforma em lista unitária
#     elif isinstance(value, list):
#         return value  # Já é uma lista, mantém
#     return [value]  # Qualquer outra coisa, transforma em lista unitária

# # Aplicar a conversão corretamente
# cols_to_convert = ['history', 'numberOfClicksHistory', 'timeOnPageHistory', 
#                    'scrollPercentageHistory', 'pageVisitsCountHistory', 'timestampHistory']

# for col in cols_to_convert:
#     df_treino[col] = df_treino[col].apply(force_list_conversion)

# # Verificar se a conversão deu certo
# print(df_treino['numberOfClicksHistory'].apply(type).value_counts())


In [10]:
# for col in cols_to_convert:
#     teste = teste.explode(col, ignore_index=True)

## Problema de Cold Start do Usuário

Os sistemas de recomendação têm um problema conhecido como **user cold start**, no qual é difícil fornecer recomendações personalizadas para usuários com nenhum ou um número muito pequeno de itens consumidos, devido à falta de informações para modelar suas preferências.

Por esse motivo, estamos mantendo no conjunto de dados apenas usuários com pelo menos 5 interações (mais de 5 notícias lidas). Com poucas interações, não há dados suficientes para entender suas preferências. Isso evita recomendações aleatórias ou baseadas apenas em popularidade.

In [11]:
df_treino = df_treino[df_treino['historySize'] >= 5]

# Verificar quantos usuários ficaram
print(f"Total de usuários após filtro: {df_treino['userId'].nunique()}")


Total de usuários após filtro: 170026


In [12]:
df_treino.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
115,2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...,Non-Logged,5,"c44af7c6-88e9-4ce2-81d6-7b8adc4e085b, 0906fe9c...","1659150541589, 1659454851970, 1659454935119, 1...","13, 11, 10, 6, 11","47348, 39297, 42652, 21185, 63258","33.36, 36.28, 30.04, 33.0, 36.22","1, 1, 1, 1, 1","1659150541589, 1659454851970, 1659454935119, 1..."
116,7804c185c0417f93cd923a48627dba045af544acdb2c50...,Non-Logged,5,"87b7c93a-809a-4709-8e59-c56b02fec7c2, f3bf3fb2...","1657413259958, 1657476222434, 1659393726063, 1...","69, 1, 6, 20, 2","410000, 10000, 30000, 100000, 10017","100.07, 11.13, 24.85, 35.2, 11.64","1, 1, 1, 1, 1","1657413259958, 1657476222434, 1659393726063, 1..."
117,64366cc3458fd2cb9dfa6837f0f1fc6f52c3c7c12d1fe9...,Non-Logged,5,"d7d7ee60-5205-42f9-a499-1e41aa036ee6, 23172c53...","1657154306452, 1658158233784, 1658697463268, 1...","33, 48, 13, 73, 30","100000, 101094, 30000, 250000, 100000","47.87, 37.88, 32.27, 55.49, 59.65","1, 1, 1, 1, 1","1657154306452, 1658158233784, 1658697463268, 1..."
118,b5916b389d764b162f7c7984ba80a0ccd5a2f14ffc16cf...,Logged,5,"ffd33f38-185a-4999-84ab-8f2db787094c, 2fdd718f...","1657037018444, 1658237578110, 1659058858061, 1...","1, 0, 1, 0, 0","71312, 177039, 90000, 174534, 46338","46.04, 52.43, 58.75, 47.96, 75.14","1, 1, 1, 1, 1","1657037018444, 1658237578110, 1659058858061, 1..."
119,7399a888f491b11d9450eac0ead878d78b9c3d6bfadf9d...,Logged,5,"ac427168-b282-443c-9d6e-8efe5b367d01, 1e4d18b9...","1657239513282, 1658870570078, 1659191441548, 1...","0, 1, 0, 6, 1","52515, 100202, 20000, 140000, 46090","21.75, 54.77, 23.79, 37.75, 46.19","1, 1, 1, 1, 1","1657239513282, 1658870570078, 1659191441548, 1..."


In [13]:
df_treino_teste = df_treino.copy()

In [14]:
df_treino_teste

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
115,2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...,Non-Logged,5,"c44af7c6-88e9-4ce2-81d6-7b8adc4e085b, 0906fe9c...","1659150541589, 1659454851970, 1659454935119, 1...","13, 11, 10, 6, 11","47348, 39297, 42652, 21185, 63258","33.36, 36.28, 30.04, 33.0, 36.22","1, 1, 1, 1, 1","1659150541589, 1659454851970, 1659454935119, 1..."
116,7804c185c0417f93cd923a48627dba045af544acdb2c50...,Non-Logged,5,"87b7c93a-809a-4709-8e59-c56b02fec7c2, f3bf3fb2...","1657413259958, 1657476222434, 1659393726063, 1...","69, 1, 6, 20, 2","410000, 10000, 30000, 100000, 10017","100.07, 11.13, 24.85, 35.2, 11.64","1, 1, 1, 1, 1","1657413259958, 1657476222434, 1659393726063, 1..."
117,64366cc3458fd2cb9dfa6837f0f1fc6f52c3c7c12d1fe9...,Non-Logged,5,"d7d7ee60-5205-42f9-a499-1e41aa036ee6, 23172c53...","1657154306452, 1658158233784, 1658697463268, 1...","33, 48, 13, 73, 30","100000, 101094, 30000, 250000, 100000","47.87, 37.88, 32.27, 55.49, 59.65","1, 1, 1, 1, 1","1657154306452, 1658158233784, 1658697463268, 1..."
118,b5916b389d764b162f7c7984ba80a0ccd5a2f14ffc16cf...,Logged,5,"ffd33f38-185a-4999-84ab-8f2db787094c, 2fdd718f...","1657037018444, 1658237578110, 1659058858061, 1...","1, 0, 1, 0, 0","71312, 177039, 90000, 174534, 46338","46.04, 52.43, 58.75, 47.96, 75.14","1, 1, 1, 1, 1","1657037018444, 1658237578110, 1659058858061, 1..."
119,7399a888f491b11d9450eac0ead878d78b9c3d6bfadf9d...,Logged,5,"ac427168-b282-443c-9d6e-8efe5b367d01, 1e4d18b9...","1657239513282, 1658870570078, 1659191441548, 1...","0, 1, 0, 6, 1","52515, 100202, 20000, 140000, 46090","21.75, 54.77, 23.79, 37.75, 46.19","1, 1, 1, 1, 1","1657239513282, 1658870570078, 1659191441548, 1..."
...,...,...,...,...,...,...,...,...,...,...
577890,02cbb677ca23c5008a9177b4260cd100519f83db1cfdfd...,Non-Logged,10,"d71e9df9-4195-4781-9658-b5225f2f71d4, 8c246d2b...","1657710041776, 1657710164288, 1657711516420, 1...","1, 0, 0, 0, 0, 0, 0, 0, 0, 1","106897, 20000, 36388, 110000, 190000, 60000, 7...","88.1, 31.86, 52.15, 72.72, 45.19, 68.95, 47.03...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1","1657710041776, 1657710164288, 1657711516420, 1..."
577891,ec727aa1c764a62a58a5a51c5a966d12af863af008a87a...,Non-Logged,10,"7ca39550-0677-4b00-b7c3-bb8ad86ce4ab, 6e958923...","1658930482752, 1659111597408, 1659111627600, 1...","0, 0, 0, 1, 1, 3, 0, 0, 0, 1","83546, 20000, 33303, 22753, 36783, 70150, 7647...","64.62, 24.56, 30.75, 35.67, 37.71, 76.72, 39.8...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1","1658930482752, 1659111597408, 1659111627600, 1..."
577892,8f75b6240c4330cb57b2c1a14d16930f3feecf6c0042d5...,Non-Logged,10,"109adb08-8ff3-408b-b828-e16b51788202, 358a38f8...","1657574243027, 1657574487044, 1658451004814, 1...","26, 8, 17, 24, 10, 4, 3, 12, 4, 13","109552, 30000, 60000, 99749, 65261, 20000, 300...","64.77, 26.73, 37.64, 54.26, 49.18, 26.05, 24.5...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1","1657574243027, 1657574487044, 1658451004814, 1..."
577893,50d57740cdd91d8571905354cfbdd5070ccfd8f2ccbad3...,Non-Logged,10,"b246d010-1bac-4550-ad00-466be2e69486, 7da17f35...","1656728038964, 1656976421767, 1657480458429, 1...","1, 0, 0, 0, 3, 0, 1, 1, 1, 0","17576, 19372, 8950, 7884, 25361, 8039, 5192, 4...","24.04, 30.04, 34.57, 15.82, 21.9, 16.04, 2.5, ...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1","1656728038964, 1656976421767, 1657480458429, 1..."


In [15]:
# Explodir a coluna 'history' separando por vírgula
df_treino_teste['history'] = df_treino_teste['history'].astype(str).str.split(",")

# Explodir os valores
df_treino_teste = df_treino_teste.explode('history')

# Remover espaços extras nos valores (se houver)
df_treino_teste['history'] = df_treino_teste['history'].str.strip()

# Verificar os resultados
print(df_treino_teste[['userId', 'history']].head())


                                                userId  \
115  2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...   
115  2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...   
115  2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...   
115  2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...   
115  2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...   

                                  history  
115  c44af7c6-88e9-4ce2-81d6-7b8adc4e085b  
115  0906fe9c-99b4-4fbf-b879-47674953717b  
115  840b6d5c-5201-453f-b106-19e1dc6d3dda  
115  608dd663-6710-4c63-a805-0034989599c5  
115  bacab81b-4dd0-4047-8d51-a9cb03a8e18e  


In [ ]:
df_treino_teste.head(6)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
115,2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...,Non-Logged,5,c44af7c6-88e9-4ce2-81d6-7b8adc4e085b,"1659150541589, 1659454851970, 1659454935119, 1...","13, 11, 10, 6, 11","47348, 39297, 42652, 21185, 63258","33.36, 36.28, 30.04, 33.0, 36.22","1, 1, 1, 1, 1","1659150541589, 1659454851970, 1659454935119, 1..."
115,2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...,Non-Logged,5,0906fe9c-99b4-4fbf-b879-47674953717b,"1659150541589, 1659454851970, 1659454935119, 1...","13, 11, 10, 6, 11","47348, 39297, 42652, 21185, 63258","33.36, 36.28, 30.04, 33.0, 36.22","1, 1, 1, 1, 1","1659150541589, 1659454851970, 1659454935119, 1..."
115,2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...,Non-Logged,5,840b6d5c-5201-453f-b106-19e1dc6d3dda,"1659150541589, 1659454851970, 1659454935119, 1...","13, 11, 10, 6, 11","47348, 39297, 42652, 21185, 63258","33.36, 36.28, 30.04, 33.0, 36.22","1, 1, 1, 1, 1","1659150541589, 1659454851970, 1659454935119, 1..."
115,2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...,Non-Logged,5,608dd663-6710-4c63-a805-0034989599c5,"1659150541589, 1659454851970, 1659454935119, 1...","13, 11, 10, 6, 11","47348, 39297, 42652, 21185, 63258","33.36, 36.28, 30.04, 33.0, 36.22","1, 1, 1, 1, 1","1659150541589, 1659454851970, 1659454935119, 1..."
115,2ee29c3b488675d9cb0bf7ec3628b3c2965bc532180334...,Non-Logged,5,bacab81b-4dd0-4047-8d51-a9cb03a8e18e,"1659150541589, 1659454851970, 1659454935119, 1...","13, 11, 10, 6, 11","47348, 39297, 42652, 21185, 63258","33.36, 36.28, 30.04, 33.0, 36.22","1, 1, 1, 1, 1","1659150541589, 1659454851970, 1659454935119, 1..."
116,7804c185c0417f93cd923a48627dba045af544acdb2c50...,Non-Logged,5,87b7c93a-809a-4709-8e59-c56b02fec7c2,"1657413259958, 1657476222434, 1659393726063, 1...","69, 1, 6, 20, 2","410000, 10000, 30000, 100000, 10017","100.07, 11.13, 24.85, 35.2, 11.64","1, 1, 1, 1, 1","1657413259958, 1657476222434, 1659393726063, 1..."


: 

In [1]:
df_treino_teste["numberOfClicksHistory"]

NameError: name 'df_treino_teste' is not defined

In [ ]:
# df_treino.to_csv("df_treino_filtrado.csv", index=False)
